<a href="https://colab.research.google.com/github/ashpakshaikh26732/Unet-FCN/blob/main/Unet_FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**all packages**

In [1]:
import tensorflow as tf
import os

In [2]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print(f'no of tpus : {strategy.num_replicas_in_sync}')
except ValueError :
  print('tpu failed to initilized')

tpu failed to initilized


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**copy datasets from drive**

In [4]:
!cp /content/drive/MyDrive/Cityscapes/gtFine_trainvaltest.zip /content/
!cp /content/drive/MyDrive/Cityscapes/leftImg8bit_trainvaltest.zip /content

**extract the data**

In [5]:
os.system("unzip -q /content/gtFine_trainvaltest.zip -d /content/ ")
os.system("unzip -q /content/leftImg8bit_trainvaltest.zip -d /content/")

256

In [6]:
print("Train Images:", len(os.listdir("/content/leftImg8bit/train/aachen")))  # Adjust path as needed
print("Train Labels:", len(os.listdir("/content/gtFine/train/aachen")))

Train Images: 174
Train Labels: 696


**Global Valrible**

In [15]:
img_height = 224
img_width = 224

**loading images and labels from directry**

In [18]:
def load_img():
    train_dir = '/content/leftImg8bit/train'
    train_image_cities = os.listdir(train_dir)
    train_images = []

    for city in train_image_cities:
        train_city = os.path.join(train_dir, city)
        images = [os.path.join(train_city, img) for img in os.listdir(train_city) if img.endswith('.png')]
        train_images.extend(images)

    return train_images

def load_labels():
    train_dir = '/content/gtFine/train'
    train_labels_cities = os.listdir(train_dir)
    train_labels = []
    for city in train_labels_cities:
        train_city_label = os.path.join(train_dir, city)
        labels = [f for f in os.listdir(train_city_label) if f.endswith('_gtFine_labelIds.png')]
        for label in labels:
            label_path = os.path.join(train_city_label, label)
            train_labels.append(label_path)
    return train_labels

**reading images from path**

In [14]:
def read_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=3)
    img = tf.image.resize(img, (img_height, img_width))
    img = img / 255.0
    return img

def read_labels(label_path):
    label = tf.io.read_file(label_path)
    label = tf.io.decode_png(label, channels=1)
    label = tf.image.resize(label, (img_height, img_width), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return tf.cast(label, tf.uint8)


**augmentation**

In [ ]:
def augment(image, label):
  if tf.random.uniform(())>0.5:
    image = tf.image.random_flip_left_right(image)
    label = tf.image.random_flip_left_right(label)

**creating dataset**

In [19]:
images = load_img()
labels = load_labels()
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.map(lambda image_path , label_path :( read_img(image_path) , label_path)).map(lambda image , label_path : (image,read_labels(label_path=label_path)))